In [1]:
import os
from pyspark.sql import SparkSession
import findspark
import numpy as np 
import pandas as pd

In [2]:
findspark.init("C:/Program Files/spark-3.5.4-bin-hadoop3")

In [3]:
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jre1.8.0_431" 
os.environ["SPARK_HOME"] = "C:/Program Files/spark-3.5.4-bin-hadoop3" 
os.environ['HADOOP_HOME '] = 'C:/Program Files/hadoop-3.4.0'

In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Spark Session") \
    .getOrCreate()
    
    

In [5]:
# Accéder au SparkContext à partir de SparkSession
sc = spark.sparkContext

# DataFrames  

A DataFrame is an immutable distributed collection of data that is organized into named columns analogous to a table in a relational database. Introduced as an experimental feature within Apache Spark 1.0 as SchemaRDD, they were renamed to DataFrames as part of the Apache Spark 1.3 release. For readers who are familiar with Python Pandas DataFrame or R DataFrame, a Spark DataFrame is a similar concept in that it allows users to easily work with structured data (for example, data tables); there are some differences as well so please temper your expectations.  

By imposing a structure onto a distributed collection of data, this allows Spark users to query structured data in Spark SQL or using expression methods (instead of lambdas). By structuring your data, this allows the Apache Spark engine – specifically, the Catalyst Optimizer – to significantly improve the performance of Spark queries. In earlier APIs of Spark (that is, RDDs), executing queries in Python could be significantly slower due to communication overhead between the Java JVM and Py4J.  





## Python to RDD communications  

Whenever a PySpark program is executed using RDDs, there is a potentially large
overhead to execute the job. As noted in the following diagram, in the PySpark
driver, the Spark Context uses Py4j to launch a JVM using the JavaSparkContext.
Any RDD transformations are initially mapped to PythonRDD objects in Java.  

Once these tasks are pushed out to the Spark Worker(s), PythonRDD objects launch
Python subprocesses using pipes to send both code and data to be processed within
Python:  

While this approach allows PySpark to distribute the processing of the data to
multiple Python subprocesses on multiple workers, as you can see, there is a lot of
context switching and communications overhead between Python and the JVM.  



## Catalyst Optimizer refresh  

As noted in Chapter 1, Understanding Spark, one of the primary reasons the Spark
SQL engine is so fast is because of the Catalyst Optimizer. For readers with a
database background, this diagram looks similar to the logical/physical planner
and cost model/cost-based optimization of a relational database management
system (RDBMS).

The significance of this is that, as opposed to immediately processing the query, the
Spark engine's Catalyst Optimizer compiles and optimizes a logical plan and has a
cost optimizer that determines the most efficient physical plan generated.  

As previously noted, the optimizer is based on functional programming
constructs and was designed with two purposes in mind: to ease the adding of new
optimization techniques and features to Spark SQL, and to allow external developers
to extend the optimizer (for example, adding data-source-specific rules, support for
new data types, and so on).  





## Speeding up PySpark with DataFrames  

The significance of DataFrames and the Catalyst Optimizer (and Project Tungsten) is
the increase in performance of PySpark queries when compared to non-optimized
RDD queries. Prior to the introduction of
DataFrames, Python query speeds were often twice as slow as the same Scala
queries using RDD. Typically, this slowdown in query performance was due
to the communications overhead between Python and the JVM.  

With DataFrames, not only was there a significant improvement in Python
performance, there is now performance parity between Python, Scala, SQL, and R.  

*[It is important to note that while, with DataFrames, PySpark is often
significantly faster, there are some exceptions. The most prominent one
is the use of Python UDFs, which results in round-trip communication
between Python and the JVM. Note, this would be the worst-case scenario
which would be similar if the compute was done on RDDs.]*  

Python can take advantage of the performance optimizations in Spark even while
the codebase for the Catalyst Optimizer is written in Scala. Basically, it is a Python
wrapper of approximately 2,000 lines of code that allows PySpark DataFrame queries
to be significantly faster.  

Altogether, Python DataFrames (as well as SQL, Scala DataFrames, and R
DataFrames) are all able to make use of the Catalyst Optimizer  



## Creating DataFrames  

Typically, you will create DataFrames by importing data using SparkSession
(or calling spark in the PySpark shell).  

First, instead of accessing the file system, we will create a DataFrame by generating
the data. In this case, we'll first create the stringJSONRDD RDD and then convert it
into a DataFrame. This code snippet creates an RDD comprised of swimmers (their
ID, name, age, and eye color) in JSON format.

### Generating our own JSON data  

Below, we will generate initially generate the stringJSONRDD RDD:

In [6]:
# Generate our own DataFrame
# This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

Now that we have created the RDD, we will convert this into a DataFrame by using
the SparkSession read.json method (that is, spark.read.json(...)). We will also
create a temporary table by using the .createOrReplaceTempView method.

* Here is the code to create a DataFrame:

In [7]:
swimmersJSON = spark.read.json(stringJSONRDD)


* Here is the code for creating a temporary table:

In [8]:
swimmersJSON.createOrReplaceTempView("swimmersJSON")

As noted in the previous chapters, many RDD operations are transformations, which
are not executed until an action operation is executed. For example, in the preceding
code snippet, the sc.parallelize is a transformation that is executed when
converting from an RDD to a DataFrame by using spark.read.json.  

It is important to note that parallelize, map, and mapPartitions are all RDD
transformations. Wrapped within the DataFrame operation, spark.read.json (in
this case), are not only the RDD transformations, but also the action which converts
the RDD into a DataFrame. This is an important call out, because even though you
are executing DataFrame operations, to debug your operations you will need to
remember that you will be making sense of RDD operations within the Spark UI.  

Note that creating the temporary table is a DataFrame transformation and not
executed until a DataFrame action is executed.  

*[DataFrame transformations and actions are similar to RDD
transformations and actions in that there is a set of operations that
are lazy (transformations). But, in comparison to RDDs, DataFrames
operations are not as lazy, primarily due to the Catalyst Optimizer.]*  

### Simple DataFrame queries  

Now that you have created the swimmersJSON DataFrame, we will be able to run
the DataFrame API, as well as SQL queries against it. Let's start with a simple query
showing all the rows within the DataFrame.

* DataFrame API query  
To do this using the DataFrame API, you can use the show(<n>) method, which
prints the first n rows to the console:

In [9]:
# DataFrame API  
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



* **SQL query**  
If you prefer writing SQL statements, you can write the following query:


In [10]:
spark.sql("SELECT * FROM swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

We are using the .collect() method, which returns all the records as a list of
Row objects. Note that you can use either the collect() or show() method for
both DataFrames and SQL queries. Just make sure that if you use .collect(),
this is for a small DataFrame, since it will return all of the rows in the DataFrame
and move them back from the executors to the driver. You can instead use
take(<n>) or show(<n>), which allow you to limit the number of rows returned
by specifying <n>.  

### Interoperating with RDDs

There are two different methods for converting existing RDDs to DataFrames (or
Datasets[T]): inferring the schema using reflection, or programmatically specifying
the schema. The former allows you to write more concise code (when your Spark
application already knows the schema), while the latter allows you to construct
DataFrames when the columns and their data types are only revealed at run time.
Note, reflection is in reference to schema reflection as opposed to Python reflection.

* **Inferring the schema using reflection**  
In the process of building the DataFrame and running the queries, we skipped over
the fact that the schema for this DataFrame was automatically defined. Initially, row
objects are constructed by passing a list of key/value pairs as **kwargs to the row
class. Then, Spark SQL converts this RDD of row objects into a DataFrame, where
the keys are the columns and the data types are inferred by sampling the data.  


`[The '**kwargs' construct allows you to pass a variable number of
parameters to a method at runtime.]` 

Going back to the code, after initially creating the swimmersJSON DataFrame,
without specifying the schema, you will notice the schema definition by using
the printSchema() method:

In [11]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



But what if we want to specify the schema because, in this example, we know that
the id is actually a long instead of a string?

* **Programmatically specifying the schema**  
In this case, let's programmatically specify the schema by bringing in Spark SQL data
types (pyspark.sql.types) and generate some .csv data for this example:

In [12]:
# Import types 
from pyspark.sql.types import * 

# Generate comma delimited data 
stringCSVRDD = sc.parallelize([
(123, 'Katie', 19, 'brown'),
(234, 'Michael', 22, 'green'),
(345, 'Simone', 23, 'blue')
])

First, we will encode the schema as a string, per the [schema] variable below. Then
we will define the schema using StructType and StructField.

In [13]:
# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
from pyspark.sql.types import IntegerType, StringType, StructField, StructType, LongType

schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

Note, the StructField class is broken down in terms of:  
• name: The name of this field  
• dataType: The data type of this field  
• nullable: Indicates whether values of this field can be null  

Finally, we will apply the schema (schema) we created to the stringCSVRDD RDD
(that is, the generated.csv data) and create a temporary view so we can query it
using SQL:

In [14]:
# Apply the schema to the RDD and create DataFrame 
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [15]:
# Create a temporary view using the dataframe
swimmers.createOrReplaceTempView("swimmers")

With this example, we have finer-grain control over the schema and can specify that
id is a long (as opposed to a string in the previous section):

In [16]:
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



### Querying with the DataFrame API

As noted in the previous section, you can start off by using collect(), show(), or
take() to view the data within your DataFrame (with the last two including the
option to limit the number of returned rows).

* **Number of rows**    
To get the number of rows within your DataFrame, you can use the count() method:

In [17]:
swimmers.count()

3

* **Running filter statements**  
To run a filter statement, you can use the filter clause; in the following code
snippet, we are using the select clause to specify the columns to be returned
as well:

In [18]:
# Get the id, age where age = 22
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [19]:
# Another way to write the above query is below
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



If we only want to get back the name of the swimmers who have an eye color
that begins with the letter b, we can use a SQL-like syntax, like, as shown in
the following code:

In [20]:
# Get the name, eyeColor where eyeColor like '%b'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



### Querying with SQL  

Let's run the same queries, except this time, we will do so using SQL queries against
the same DataFrame. Recall that this DataFrame is accessible because we executed
the .createOrReplaceTempView method for swimmers.

* **Number of rows**  
The following is the code snippet to get the number of rows within your DataFrame
using SQL:

In [21]:
spark.sql("SELECT COUNT(1) FROM swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



* **Running filter statements using the where Clauses**  

To run a filter statement using SQL, you can use the where clause, as noted in the
following code snippet:

In [23]:
spark.sql("SELECT id, age FROM swimmers WHERE age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



As with the DataFrame API querying, if we want to get back the name of the
swimmers who have an eye color that begins with the letter b only, we can use
the like syntax as well:

In [24]:
spark.sql(
    "SELECT name, eyeColor FROM swimmers WHERE eyeColor LIKE '%b'").show()

+----+--------+
|name|eyeColor|
+----+--------+
+----+--------+



`*An important note when working with Spark SQL and DataFrames is that
while it is easy to work with CSV, JSON, and a variety of data formats,
the most common storage format for Spark SQL analytics queries is the
Parquet file format. It is a columnar format that is supported by many
other data processing systems and Spark SQL supports both reading
and writing Parquet files that automatically preserves the schema of
the original data.*`

## DataFrame scenario – on-time flight performance  

To showcase the types of queries you can do with DataFrames, let's look at the use
case of on-time flight performance. We will analyze the Airline On-Time Performance
and Causes of Flight Delays: On-Time Data (http://bit.ly/2ccJPPM), and join this
with the airports dataset, obtained from the Open Flights Airport, airline, and route
data (http://bit.ly/2ccK5hw), to better understand the variables associated with
flight delays.

* **Preparing the source datasets**  
We will first process the source airports and flight performance datasets by
specifying their file path location and importing them using SparkSession:

In [25]:
# Set File Paths
flightPerfFilePath ="datasets/departuredelays.csv"
airportsFilePath ="datasets/airport-codes-na.txt"

# Obtain Airports dataset
airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

# Obtain Departure Delays dataset
flightPerf = spark.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

# Cache the Departure Delays dataset
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]


Note that we're importing the data using the CSV reader (com.databricks.spark.
csv), which works for any specified delimiter (note that the airports data is tabdelimited,
while the flight performance data is comma-delimited). Finally, we cache
the flight dataset so subsequent queries will be faster.

* **Joining flight performance and airports**  

One of the more common tasks with DataFrames/SQL is to join two different
datasets; it is often one of the more demanding operations (from a performance
perspective). With DataFrames, a lot of the performance optimizations for these
joins are included by default:

In [26]:
# Query Sum of Flight Delays by City and Origin Code (for Washington State)
spark.sql("""
          SELECT a.City, f.origin, SUM(f.delay) AS Delays
          FROM FlightPerformance f
          JOIN airports a
          ON a.IATA = f.origin
          WHERE a.State = 'WA'
          GROUP BY a.City, f.origin
          ORDER BY SUM(f.delay) DESC""").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In our scenario, we are querying the total delays by city and origin code for the state
of Washington. This will require joining the flight performance data with the airports
data by International Air Transport Association (IATA) code.

* **Visualizing our flight-performance data**  
Let's continue visualizing our data, but broken down by all states in the continental US:

In [27]:
## Query Sum of Flight Delays by State (for the US)
spark.sql("""
          SELECT a.State, SUM(f.delay) as Delays
          FROM FlightPerformance f
          JOIN airports a
          ON a.IATA = f.origin
          WHERE a.Country = 'USA'
          GROUP BY a.state""").show()

+-----+---------+
|State|   Delays|
+-----+---------+
|   SC|  80666.0|
|   LA| 199136.0|
|   MN| 256811.0|
|   NJ| 452791.0|
|   OR| 109333.0|
|   VA|  98016.0|
| NULL| 397237.0|
|   WY|  15365.0|
|   MI| 366486.0|
|   NV| 474208.0|
|   WI| 152311.0|
|   ID|  22932.0|
|   CA|1891919.0|
|   CT|  54662.0|
|   MT|  19271.0|
|   NC| 394256.0|
|   VT|  14755.0|
|   MD| 362845.0|
|   IL|1630792.0|
|   ME|  15214.0|
+-----+---------+
only showing top 20 rows



One of the key benefits of DataFrames is that the information is structured similar to
a table. Therefore, whether you are using notebooks or your favorite BI tool, you will
be able to quickly visualize your data.

## Spark Dataset API  
After this discussion about Spark DataFrames, let's have a quick recap of the Spark
Dataset API. Introduced in Apache Spark 1.6, the goal of Spark Datasets was to
provide an API that allows users to easily express transformations on domain
objects, while also providing the performance and benefits of the robust Spark SQL
execution engine. As part of the Spark 2.0 release (and as noted in the diagram
below), the DataFrame APIs is merged into the Dataset API thus unifying data
processing capabilities across all libraries. Because of this unification, developers
now have fewer concepts to learn or remember, and work with a single high-level
and type-safe API – called Dataset:

Conceptually, the Spark DataFrame is an alias for a collection of generic objects
Dataset[Row], where a Row is a generic untyped JVM object. Dataset, by contrast, is
a collection of strongly-typed JVM objects, dictated by a case class you define, in Scala
or Java. This last point is particularly important as this means that the Dataset API
is not supported by PySpark due to the lack of benefit from the type enhancements.
Note, for the parts of the Dataset API that are not available in PySpark, they can be
accessed by converting to an RDD or by using UDFs.

## Summary  
With Spark DataFrames, Python developers can make use of a simpler abstraction
layer that is also potentially significantly faster. One of the main reasons Python is
initially slower within Spark is due to the communication layer between Python
sub-processes and the JVM. For Python DataFrame users, we have a Python wrapper
around Scala DataFrames that avoids the Python sub-process/JVM communication
overhead. Spark DataFrames has many performance enhancements through the
Catalyst Optimizer and Project Tungsten which we have reviewed in this chapter.  

In this chapter, we also reviewed how to work with Spark DataFrames and worked
on an on-time flight performance scenario using DataFrames.
In this chapter, we created and worked with DataFrames by generating the data or
making use of existing datasets.  

In the next chapter, we will discuss how to transform and understand your
own data.